In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
import streamlit as st

In [3]:
# Function to load stock data
def load_data(ticker, start_date='2010-01-01', end_date='2023-01-01'):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

# Function to preprocess data
def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data['Close'].values.reshape(-1, 1))
    
    # Define sequence length
    sequence_length = 60
    X, y = [], []

    # Create sequences of length 60
    for i in range(sequence_length, len(data_scaled)):
        X.append(data_scaled[i-sequence_length:i, 0])
        y.append(data_scaled[i, 0])

    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    
    # Split into training and test sets (80-20 split)
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]
    
    return X_train, y_train, X_test, y_test, scaler


In [4]:
# Function to build the GRU model
def build_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(GRU(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Output layer
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [5]:
# Function to train the model
def train_model(model, X_train, y_train, epochs=20, batch_size=32):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1)
    return model

In [6]:
# Function to predict the next day price based on the most recent data
def predict_next_day_price(model, data, scaler):
    # Use the last 60 days for prediction
    last_60_days = data['Close'].values[-60:].reshape(-1, 1)
    last_60_days_scaled = scaler.transform(last_60_days)
    X_predict = np.array([last_60_days_scaled])
    X_predict = np.reshape(X_predict, (X_predict.shape[0], X_predict.shape[1], 1))

    # Predict and inverse scale
    predicted_price = model.predict(X_predict)
    predicted_price = scaler.inverse_transform(predicted_price)
    return predicted_price[0][0]
